In [26]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [28]:
START_YEAR = 2009
END_YEAR = datetime.datetime.now().year - 1

In [29]:
def collect_singles_overview_data(year, day):
    url = f"https://www.atptour.com/en/rankings/singles?rankRange=1-5000&rankDate={year}-12-{day}"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    ranking_table = soup.find("table", {"class": "mega-table"})

    # return if there is no table meaning that the date is not a Monday
    if ranking_table is None:
        return False

    rows = ranking_table.find_all("tr")

    # return if there is no data meaning that the date is not a Monday
    if len(rows) == 1:
        return False

    singles = []
    for row in rows[1:]:
        cells = row.find_all("td")

        ranking = int(cells[0].text.strip().replace("T", ""))
        country = cells[2].find("img")["alt"]
        name = cells[3].text.strip()
        player_id = cells[3].find("a")["href"].split("/")[4]
        link = "https://www.atptour.com" + cells[3].find("a")["href"]
        age = cells[4].text.strip()
        if age == "":
            age = None
        else:
            age = int(age)
        points = int(cells[5].text.strip().replace(",", ""))
        tournaments_played = int(cells[7].text.strip())

        singles.append({
            "ranking": ranking,
            "country": country,
            "name": name,
            "player_id": player_id,
            "link": link,
            "age": age,
            "points": points,
            "tournaments_played": tournaments_played
        })

    df = pd.DataFrame(singles)
    df.to_csv(f"data/overview/singles/{str(year)}.gz", index=False, compression="gzip")
    return True

In [30]:
def collect_doubles_overview_data(year, day):
    url = f"https://www.atptour.com/en/rankings/doubles?rankRange=1-5000&rankDate={year}-12-{day}"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    ranking_table = soup.find("table", {"class": "mega-table"})
    rows = ranking_table.find_all("tr")

    doubles = []
    for row in rows[1:]:
        cells = row.find_all("td")

        ranking = int(cells[0].text.strip().replace("T", ""))
        country = cells[2].find("img")["alt"]
        name = cells[3].text.strip()
        player_id = cells[3].find("a")["href"].split("/")[4]
        link = "https://www.atptour.com" + cells[3].find("a")["href"]
        age = cells[4].text.strip()
        if age == "":
            age = None
        else:
            age = int(age)
        points = int(cells[5].text.strip().replace(",", ""))
        tournaments_played = int(cells[7].text.strip())

        doubles.append({
            "ranking": ranking,
            "country": country,
            "name": name,
            "player_id": player_id,
            "link": link,
            "age": age,
            "points": points,
            "tournaments_played": tournaments_played
        })

    df = pd.DataFrame(doubles)
    df.to_csv(f"data/overview/doubles/{str(year)}.gz", index=False, compression="gzip")

In [32]:
# loop through all years
for year in range(START_YEAR, END_YEAR + 1):
    # find the last Monday of the year
    for day in range(31, 24, -1):
        success = collect_singles_overview_data(year, day)
        if not success:
            continue

        collect_doubles_overview_data(year, day)
        break

In [36]:
def collect_singles_activity_data(year):
    singles = pd.read_csv(f"data/overview/singles/{str(year)}.gz")

    players = []
    for _, row in singles.iterrows():
        activity_link = row["link"].replace("overview", "player-activity") + "?year=" + str(year) + "&matchType=singles"
        response = requests.get(activity_link)
        soup = BeautifulSoup(response.text, "html.parser")

        activity_table = soup.find("div", {"data-filtered-module": "playerActivityTables"})
        if activity_table is None:
            continue

        tournaments = activity_table.find_all("div", {"class": "activity-tournament-table"})

        tournaments_played = []
        for tourney in tournaments:
            tournament_info = tourney.find("table", {"class": "tourney-results-wrapper"})
            tournament_type = tournament_info.find("img")["alt"]

            match_info = tourney.find("table", {"class": "mega-table"})
            match_rows = match_info.find_all("tr")

            wins = 0
            losses = 0
            for match in match_rows:
                result = match.find_all("td")[3].text.strip()
                if result == "W":
                    wins += 1
                elif result == "L":
                    losses += 1
            
            other_info = tourney.find("div", {"class": "activity-tournament-caption"}).text.strip()
            other_info_split = other_info.split(", ")

            points = 0
            prize_money = ""
            for info in other_info_split:
                if "Points" in info:
                    points = int(info.split(" ")[-1])
                elif "Prize Money" in info:
                    prize_money = info.split(" ")[-1].replace(",", "")

            tournaments_played.append({
                "tournament_type": tournament_type,
                "wins": wins,
                "losses": losses,
                "points": points,
                "prize_money": prize_money
            })
        
        # include all other player info as well
        players.append({
            "ranking": row["ranking"],
            "country": row["country"],
            "name": row["name"],
            "player_id": row["player_id"],
            "link": row["link"],
            "age": row["age"],
            "points": row["points"],
            "tournament_played": row["tournaments_played"],
            "tournaments": tournaments_played
        })

    df = pd.DataFrame(players)
    df.to_csv(f"data/activity/singles/{str(year)}.gz", index=False, compression="gzip")

In [37]:
def collect_doubles_activity_data(year):
    doubles = pd.read_csv(f"data/overview/doubles/{str(year)}.gz")

    players = []
    for _, row in doubles.iterrows():
        activity_link = row["link"].replace("overview", "player-activity") + "?year=" + str(year) + "&matchType=doubles"
        response = requests.get(activity_link)
        soup = BeautifulSoup(response.text, "html.parser")

        activity_table = soup.find("div", {"data-filtered-module": "playerActivityTables"})
        if activity_table is None:
            continue

        tournaments = activity_table.find_all("div", {"class": "activity-tournament-table"})

        tournaments_played = []
        for tourney in tournaments:
            tournament_info = tourney.find("table", {"class": "tourney-results-wrapper"})
            tournament_type = tournament_info.find("img")["alt"]

            match_info = tourney.find("table", {"class": "mega-table"})
            match_rows = match_info.find_all("tr")

            wins = 0
            losses = 0
            for match in match_rows:
                result = match.find_all("td")[3].text.strip()
                if result == "W":
                    wins += 1
                elif result == "L":
                    losses += 1
            
            other_info = tourney.find("div", {"class": "activity-tournament-caption"}).text.strip()
            other_info_split = other_info.split(", ")

            points = 0
            prize_money = ""
            for info in other_info_split:
                if "Points" in info:
                    points = int(info.split(" ")[-1])
                elif "Prize Money" in info:
                    prize_money = info.split(" ")[-1].replace(",", "")

            tournaments_played.append({
                "tournament_type": tournament_type,
                "wins": wins,
                "losses": losses,
                "points": points,
                "prize_money": prize_money
            })
        
        # include all other player info as well
        players.append({
            "ranking": row["ranking"],
            "country": row["country"],
            "name": row["name"],
            "player_id": row["player_id"],
            "link": row["link"],
            "age": row["age"],
            "points": row["points"],
            "tournament_played": row["tournaments_played"],
            "tournaments": tournaments_played
        })

    df = pd.DataFrame(players)
    df.to_csv(f"data/activity/doubles/{str(year)}.gz", index=False, compression="gzip")

In [ ]:
# loop through all years
for year in range(START_YEAR, END_YEAR + 1):
    collect_singles_activity_data(year)
    collect_doubles_activity_data(year)